In [1]:
!pip install bitsandbytes transformers einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 10.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.2 MB/s eta 0:00:00


In [2]:
import torch
import random
import requests
import pandas as pd
from PIL import Image, ImageFilter
from io import BytesIO
import matplotlib.pyplot as plt
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
df = pd.read_csv("/kaggle/input/test-main/test.csv")
df = df[df["entity_name"] == "wattage"]
df

,index,image_link,group_id,entity_name
18,18,https://m.media-amazon.com/images/I/218BCzgKxu...,348551,wattage
38,38,https://m.media-amazon.com/images/I/21cLufe8Y5...,648011,wattage
52,52,https://m.media-amazon.com/images/I/21pFwZ8ghk...,955292,wattage
59,59,https://m.media-amazon.com/images/I/21tOerzlVM...,957050,wattage
74,74,https://m.media-amazon.com/images/I/31+1xiVjZB...,997176,wattage
...,...,...,...,...
131100,131201,https://m.media-amazon.com/images/I/91wic4tk-b...,219211,wattage
131105,131206,https://m.media-amazon.com/images/I/91x1yoXFpQ...,219211,wattage
131108,131209,https://m.media-amazon.com/images/I/91x2QhRqmv...,219211,wattage
131125,131226,https://m.media-amazon.com/images/I/91xp-halWC...,983323,wattage


In [4]:
model_id = "vikhyatk/moondream2"
revision = "2024-08-26"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    revision=revision,
    trust_remote_code=True,
    torch_dtype=torch.float16,
).eval().to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_id, revision=revision)

config.json:   0%|          | 0.00/319 [00:00<?, ?B/s]

configuration_moondream.py:   0%|          | 0.00/3.34k [00:00<?, ?B/s]

moondream.py:   0%|          | 0.00/7.20k [00:00<?, ?B/s]

modeling_phi.py:   0%|          | 0.00/63.1k [00:00<?, ?B/s]

region_model.py:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [5]:
df["entity_value"] = ""
df

,index,image_link,group_id,entity_name,entity_value
18,18,https://m.media-amazon.com/images/I/218BCzgKxu...,348551,wattage,
38,38,https://m.media-amazon.com/images/I/21cLufe8Y5...,648011,wattage,
52,52,https://m.media-amazon.com/images/I/21pFwZ8ghk...,955292,wattage,
59,59,https://m.media-amazon.com/images/I/21tOerzlVM...,957050,wattage,
74,74,https://m.media-amazon.com/images/I/31+1xiVjZB...,997176,wattage,
...,...,...,...,...,...
131100,131201,https://m.media-amazon.com/images/I/91wic4tk-b...,219211,wattage,
131105,131206,https://m.media-amazon.com/images/I/91x1yoXFpQ...,219211,wattage,
131108,131209,https://m.media-amazon.com/images/I/91x2QhRqmv...,219211,wattage,
131125,131226,https://m.media-amazon.com/images/I/91xp-halWC...,983323,wattage,


In [7]:
from tqdm import tqdm

In [ ]:
total_processed = 0

# rows after 786
for i, row in tqdm(df.iterrows(), total=len(df)):
    img = Image.open(requests.get(row["image_link"], stream=True).raw)
#     img = img.convert("L")
#     img = img.point(lambda p: p * 1.2)
#     img = img.filter(ImageFilter.MinFilter(3))
    enc_image = model.encode_image(img).to('cuda')
    prediction = model.answer_question(enc_image, "wattage of the appliance?", tokenizer)
    
    df.at[i, "entity_value"] = prediction
    total_processed += 1
    
    if total_processed % 20 == 0:
        df.to_csv(
            "test_todof_with_prediction3.csv",
            index=False,
        )

 30%|███       | 1643/5447 [13:44<42:05,  1.51it/s] 